# Germinal center:  
---
## Part1: Dataset generation


Data (from Claudio, IFOM): 
10 images depicting germinal centers (annotated by hand) stained for Dapi, CD3 and AICDA. 

Aims:

The first step is to generate the dataset that we will use for downstream modelling and analysis. 

1. preprocess the images such that we have 8bit tiff files
2. perform nuclear segmentation using pretrained stardist model
3. measure nuclear morphology and chromatin organization features (chrommetrics) of each nucleus
4. identify cell boundary by expanding the nuclear border
5. measure cellular levels of each protein 
6. measure the spatial location of each nucleus in a image


### Setting up the environment 

Read in the libraries and the set up the paths

In [1]:
# import libraries
import sys

sys.path.append("../..")

from pathlib import Path
from glob import glob
import pandas as pd
import os
from tqdm.notebook import tqdm

from src.batch.nuclear_segmentation import segment_objects_stardist2d
from src.batch.extract_features import (
    extract_nmco_feats_batch,
    measure_intensity_batch,
    extract_spatial_coordinates_batch,
)
from src.batch.cell_segmentation import cell_seg_dilation_batch
from src.utils.preprocess_images import (
    extract_channel_save_image,
    quantile_normalize_and_save_images,
)
from src.utils.cell_type_detection import assign_cell_status
from src.utils.data_viz import visualize_segmentation_results

%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
root_dir = "../../data/dataset1/images"
raw_image_dir = os.path.join(root_dir, "raw")
gc_mask_dir = os.path.join(root_dir, "germinal_center_anno")

merged_image_dir = os.path.join(raw_image_dir, "merged")
dna_image_dir = os.path.join(raw_image_dir, "dapi")
cd3_image_dir = os.path.join(raw_image_dir, "cd3")
aicda_image_dir = os.path.join(raw_image_dir, "aicda")

scaled_dna_image_dir = os.path.join(raw_image_dir, "dapi_scaled")

segmented_nuclei_dir = os.path.join(root_dir, "segmented_nucleus")
nuclei_rois_dir = os.path.join(root_dir, "segmented_nuclei_ijroi")
nuclear_features_dir = os.path.join(root_dir, "chrometric_features")
segmented_cells_dir = os.path.join(root_dir, "segmented_cells")

cellular_aidca_levels_dir = os.path.join(root_dir, "aicda_level")
cellular_cd3_levels_dir = os.path.join(root_dir, "cd3_level")
germinal_center_loc_dir = os.path.join(root_dir, "position_wrt_germinal_center")

spatial_cordiates_dir = os.path.join(root_dir, "spatial_cordiates")
consolidated_features_dir = os.path.join(root_dir, "consolidated_features")

<IPython.core.display.Javascript object>

### Compute features

We process the channels, segment nuclei using a pretrained stardist model, extract nmco features and segment cells by expanding nuclear boundary and then measure cellular levels of AICDA, CD3 and location within the germinal center. 

In [3]:
# process the image rgb to single channel tiff
print("Preprocessing image to make single channel tiff images......")
extract_channel_save_image(merged_image_dir, dna_image_dir, 1)
extract_channel_save_image(merged_image_dir, cd3_image_dir, 2)
extract_channel_save_image(merged_image_dir, aicda_image_dir, 3)

Preprocessing image to make single channel tiff images......


<IPython.core.display.Javascript object>

In [4]:
# quantile normalize dapi images
quantile_normalize_and_save_images(dna_image_dir, scaled_dna_image_dir, mask_dir =gc_mask_dir,  quantiles=[0.01, 0.998])


<IPython.core.display.Javascript object>

In [8]:
# perform nuclear segmentation 
print('Nuclear segmentation........')
segment_objects_stardist2d(image_dir = scaled_dna_image_dir+"/",
                               output_dir_labels = segmented_nuclei_dir+"/",
                               output_dir_ijroi = nuclei_rois_dir+"/",
                               use_pretrained = True,
                               prob_thresh=0.43)
                               #normalize_quants=[1, 99.8])



Nuclear segmentation........
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
['../../data/dataset1/images/raw/dapi_scaled/1.tif', '../../data/dataset1/images/raw/dapi_scaled/10.tif', '../../data/dataset1/images/raw/dapi_scaled/14.tif', '../../data/dataset1/images/raw/dapi_scaled/2.tif', '../../data/dataset1/images/raw/dapi_scaled/3.tif', '../../data/dataset1/images/raw/dapi_scaled/4.tif', '../../data/dataset1/images/raw/dapi_scaled/5.tif', '../../data/dataset1/images/raw/dapi_scaled/6.tif', '../../data/dataset1/images/raw/dapi_scaled/7.tif', '../../data/dataset1/images/raw/dapi_scaled/8.tif']
1/1 [==============================] - 0s 14ms/step


  0%|                                                                                                                                                                                                    | 0/42 [00:00<?, ?it/s]

1/1 [==============================] - 0s 127ms/step


  2%|████▍                                                                                                                                                                                       | 1/42 [00:00<00:09,  4.17it/s]

1/1 [==============================] - 0s 15ms/step


...

1/1 [==============================] - 0s 14ms/step


 97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊      | 29/30 [00:03<00:00,  9.78it/s]

1/1 [==============================] - 0s 15ms/step


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:03<00:00,  9.63it/s]


<IPython.core.display.Javascript object>

In [9]:
visualize_segmentation_results(
    image_dir=scaled_dna_image_dir,
    mask_dir=segmented_nuclei_dir,
    overlay_output_dir=os.path.join(root_dir, "segmentation_visualization", "overlay"),
    outline_output_dir=os.path.join(root_dir, "segmentation_visualization", "outline"),
    alpha=0.3,
)

<IPython.core.display.Javascript object>

In [ ]:
# compute nuclear features
print("Nuclear feature extraction........")
nuc_features = extract_nmco_feats_batch(
    raw_image_path=scaled_dna_image_dir + "/",
    labelled_image_path=segmented_nuclei_dir + "/",
    output_dir=nuclear_features_dir + "/",
)

In [ ]:
# segment cells by boundary expansion
print('Nuclear boundary expansion to segment cells.......')
cell_seg_dilation_batch(segmented_nuclei_dir+"/", segmented_cells_dir+"/")


In [ ]:
# measure cellular levels of proteins
print("computing cellular levels of proteins.....")

cd3_levels = measure_intensity_batch(
    labelled_image_path=segmented_cells_dir + "/",
    protein_image_path=cd3_image_dir + "/",
    output_dir=cellular_cd3_levels_dir + "/",
)

aicda_levels = measure_intensity_batch(
    labelled_image_path=segmented_cells_dir + "/",
    protein_image_path=aicda_image_dir + "/",
    output_dir=cellular_aidca_levels_dir + "/",
)

gc_levels = measure_intensity_batch(
    labelled_image_path=segmented_cells_dir + "/",
    protein_image_path=gc_mask_dir + "/",
    output_dir=germinal_center_loc_dir + "/",
)

In [ ]:
# measure spatial nuclear location
spatial_coordiates = extract_spatial_coordinates_batch(labelled_image_path = segmented_nuclei_dir+"/", 
                        output_dir = spatial_cordiates_dir+"/")


### Save output

In [ ]:
# setup unique nuclear ids
nuc_features['nuc_id'] = nuc_features['image'].astype(str) + '_'+ nuc_features['label'].astype(str)
aicda_levels['nuc_id'] = aicda_levels['image'].astype(str) + '_'+ aicda_levels['label'].astype(str)
cd3_levels['nuc_id'] = cd3_levels['image'].astype(str) + '_'+ cd3_levels['label'].astype(str)
gc_levels['nuc_id'] = gc_levels['image'].astype(str) + '_'+ gc_levels['label'].astype(str)

# save the data aquired
Path(consolidated_features_dir).mkdir(parents=True, exist_ok=True)

nuc_features.to_csv(consolidated_features_dir+"/" +"nuc_features.csv")
aicda_levels.to_csv(consolidated_features_dir+"/" +"aicda_levels.csv")
cd3_levels.to_csv(consolidated_features_dir+"/" +"cd3_levels.csv")
gc_levels.to_csv(consolidated_features_dir+"/" +"gc_levels.csv")
spatial_coordiates.to_csv(consolidated_features_dir+"/" +"spatial_coordiates.csv")
